<a href="https://colab.research.google.com/github/hunterkwalt/BRD_project/blob/main/Pairwise_Bact_Zhang_Metric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

import pandas as pd
print(pd.__version__)
import csv
from google.colab import sheets
from itertools import combinations


Python 3.12.12
2.2.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#grabs csv file from personal Google Drive
data = pd.read_csv('/content/drive/MyDrive/co_occur_with_bacteria_onehot.csv')
data

Mounted at /content/drive


,Unnamed: 0,SRA,Virus,Presence/Absence
0,1,SRR11836760,BCoV,0
1,2,SRR11836760,BEVE,1
2,3,SRR11836760,BHKV,0
3,4,SRR11836760,BHV1,0
4,5,SRR11836760,BNV,0
...,...,...,...,...
10768,10769,SRR11837040,Moraxella,0
10769,10770,SRR11837040,Mannheimia,0
10770,10771,SRR11837040,Pasteurella,0
10771,10772,SRR11837040,Histophilus,0


In [ ]:
def zhangs_rule(df, Dict, pathogen1, pathogen2):
    #finds number of columns where pathogens co-occurr (i.e. support for pathogen1 union pathogen2)
    supportAC = len(df[(df[pathogen1]== 1) & (df[pathogen2]== 1)])/ len(df)
    supportA = float(Dict[pathogen1])
    supportC = float(Dict[pathogen2])
    numerator = supportAC - (supportA * supportC)
    #denominator for comparison: if pathogen1, then pathogen2
    forDenominator = max(supportAC*(1-supportA), supportA *(supportC-supportAC))
    #denominator for comparison: if pathogen2, then pathogen1
    revDenominator = max(supportAC*(1-supportC), supportC *(supportA-supportAC))
    #since demoninator=0 will cause mathematical error, will set default output to 0 (i.e. no significant association)
    out1 = 0
    out2 = 0
    #will calculate zhang metric as long as there is some association (+ or -)
    if forDenominator != 0:
      out1 = numerator/forDenominator;
    if revDenominator != 0:
      out2 = numerator/revDenominator
    return (out1,out2, supportA, supportC, supportAC)

In [ ]:
basket_encoded = (data.groupby(['SRA', 'Virus'])["Presence/Absence"].sum().unstack().reset_index().fillna(0).set_index('SRA'))
supportDict={}
#dropping pathogens BVDV2 and BVDV3 bc i was told to idk why
basket_encoded= basket_encoded.drop(axis=1, labels= ["BVDV2","BVDV3"])
#dropping low abundance pathogens from data
for col in basket_encoded.columns:
  if basket_encoded[col].mean() >= 0.0001:
    print(col ,basket_encoded[col].mean())
    #adds individual pathogen support values to dictionary
    supportDict.update({col:basket_encoded[col].mean()})

In [ ]:
#creates a list with all pairwise pathogen comparisons
supportList = list(combinations(supportDict.keys(), 2))

In [ ]:
zhangSeries = {}
for i in range(len(supportList)):
  #adds zhang metric to dataframe
  zhangSeries[supportList[i]] = zhangs_rule(basket_encoded, supportDict, supportList[i][0], supportList[i][1])

In [ ]:
#converts series into dataframe
test = pd.DataFrame(zhangSeries)
#transforms dataframe
test = test.T
#adds column names to dataframe
test.columns = ["Zhang(A —> B)", "Zhang(B —> A)", "Support A", "Support B", "(A U B)"]
#export zhang metric data to csv file
test.to_csv("zhangs_test.csv")

In [ ]:
test


Zhang(A —> B)  Zhang(B —> A)  Support A  Support B  \
BCoV       BEVE              0.666816       0.475890   0.410935   0.174603   
           BHKV              0.228963       0.145111   0.410935   0.070547   
           BHV1              0.767465       0.482737   0.410935   0.063492   
           BNV              -0.362899      -0.260171   0.410935   0.045855   
           BPIV3             0.505863       0.321214   0.410935   0.072310   
...                               ...            ...        ...        ...   
Mannheimia Mycoplasma        0.484733       0.590698   0.537919   0.620811   
           Pasteurella       0.781145       0.758000   0.537919   0.523810   
Moraxella  Mycoplasma        0.477464       0.415282   0.670194   0.620811   
           Pasteurella       0.796791       0.551852   0.670194   0.523810   
Mycoplasma Pasteurella       0.652511       0.519592   0.620811   0.523810   

                         (A U B)  
BCoV       BEVE         0.118166  
           BHKV         0.033510  
           BHV1         0.047619  
           BNV          0.014109  
           BPIV3        0.042328  
...                          ...  
Mannheimia Mycoplasma   0.430335  
           Pasteurella  0.440917  
Moraxella  Mycoplasma   0.493827  
           Pasteurella  0.476190  
Mycoplasma Pasteurella  0.432099  

[136 rows x 5 columns]